<a href="https://colab.research.google.com/github/nneibaue/ocean_explorer/blob/staging/explorer_official.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Ocean Data Explorer</h1>

In [1]:
# etsp stuff
import ocean
import plotting as op
import ocean_utils as utils
from IPython.core.display import display, HTML

# Convenience function to reload github and reimport

from importlib import reload
def refresh_modules():
  reload(ocean)
  reload(op)
  reload(utils)

display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

# Data Import

In [2]:
refresh_modules()
EXPERIMENT_DIR = "deglitched_profiles"
ELEMENTS_OF_INTEREST = "Br,Ca,Cu,Fe,K,Cl,Mn,S,Si,Zn"
ELEMENTS_OF_INTEREST=ELEMENTS_OF_INTEREST.split(',')
ORBITALS = "K"
profiles = ocean.load_profiles(EXPERIMENT_DIR,
                               elements_of_interest=ELEMENTS_OF_INTEREST,
                               orbitals=[ORBITALS],
                               normalized=True)

ETSP_PS2: Successfully imported data for 750m
ETSP_PS2: Successfully imported data for 250m
ETSP_PS2: Successfully imported data for 400m
ETSP_PS2: Successfully imported data for 20m
ETSP_PS2: Successfully imported data for 80m
ETSP deglitch.xlsx is not a valid name for a Depth!
ETSP_PS2: Successfully imported data for 40m
ETSP_PS3: Successfully imported data for 165m
ETSP_PS3: Successfully imported data for 10m
ETSP_PS3: Successfully imported data for 50m
ETSP_PS3: Successfully imported data for 250m
ETSP_PS3: Successfully imported data for 900m
ETSP deglitch.xlsx is not a valid name for a Depth!
ETSP_PS3: Successfully imported data for 25m
ETSP_PS3: Successfully imported data for 40m
ETSP_PS3: Successfully imported data for 65m


# UI

In [31]:
refresh_modules()
op.run_ui(profiles, op.noise_flagging_ui, title='Noise Flagging')

In [4]:
op.run_ui(profiles, lambda profile: op.ribbon_plot_ui(profile, ELEMENTS_OF_INTEREST), title='Ribbon Plot')

In [11]:
op.run_ui(profiles, lambda profile: op.image_ui(profile, ELEMENTS_OF_INTEREST), title='Image Explorer')

In [10]:
refresh_modules()

In [9]:
import ipywidgets as iw
sliders = [iw.FloatSlider(min=0, max=5, step=1) for _ in range(10)]
def print_value(val):
    print(val['new'])
    
for slider in sliders:
    slider.observe(print_value, names='value')

#display(slider)
tabs = iw.Tab(children=sliders)
display(tabs)

In [28]:
import numpy as np
p = profiles['ETSP_PS2']
d = p.depths[-1]
s = d.scans[0]
s.data['Br'].sum()
#s.get_unique_groups(['Cu'], 'Cu_counts')
#np.nansum(det.data)
help(op.ribbon_plot)


Help on function ribbon_plot in module plotting:

ribbon_plot(profile, element_filter, filter_by='Cu', combine_scans=True, combine_detsums=True, N=8, normalize_by='counts', base64=False)
    Shows fractional concentration of element among different groups.
    
    Args:
      profile: ocean.Profile object
      element_filter: dict specifying how elements should be filtered. For more 
        info, see `Depth.apply_element_filter`. In Colab, this can be done by
        opening a new cell and running `Depth.apply_element_filter?`.
      combine_scans: bool. Whether or not to show individual scans as different
        bars in the ribbon plot. If this is True, then scans will be combined
      combine_detsums: bool. Whether or not to combine detsums when applying the
        filter functions defined in `element_filter`. If this is True, then all
        detsums within a given depth will be considered when applying the filter.
      N: int. The amount of groups to use. For example, N=12 m

In [18]:
widgets = [op.DetsumPlot(d).widget for _ in range(4)]
iw.HBox(widgets)